In [ ]:
import matplotlib.animation as animation
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt 
from IPython.display import HTML

from double_pendulum import Simulation, RK4, SSPRK3, init, animate
from double_pendulum import acceleration_double_pendulum, acceleration_n_pendulum


In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['animation.embed_limit'] = 5*1e7

In [ ]:
state = [2., 2.5, 0., 0.]
M = [1., 1.]
L = [1., 1.]
kwargs = { "M": M, "L": L, 'trail_frames': 50}


sim = Simulation( state, acceleration_double_pendulum, SSPRK3, kwargs )
fig = init(sim)

anim = animation.FuncAnimation(fig, animate, frames=2000,                       
            interval=30, blit=True, fargs=(0.0175, sim))
plt.close()

This generates a relatively compact MPEG video from the HTML5 animation

In [ ]:
HTML(anim.to_html5_video())

Plots the animation directly using an interactive HTML5 widget (a bit more memory intensive) <br>
The 'embed_limit' set maximum memory usage in bytes, set to 50 MB by default

In [ ]:
HTML(anim.to_jshtml())

Optionally, we can save the animation in GIF format

In [ ]:
anim.save('example.gif', writer='imagemagick', fps=30)

In [ ]:
N = 20

state = [ 1 + 0.33*(n+1)/float(N) for n in range(N) ] + [0.] * N
M = [1.] * N
L = [1.] * N
kwargs = { "M": M, "L": L, 'trail_frames': 50}


sim = Simulation( state, acceleration_n_pendulum, SSPRK3, kwargs )
fig = init(sim)

anim = animation.FuncAnimation(fig, animate, frames=3000,                       
            interval=3, blit=True, fargs=(0.005, sim))
plt.close()

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
anim.save('example_20n.gif', writer='imagemagick', fps=60)